In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

tb_logdir = "/tmp/tf_tutorial/06_ConvNeuralNetwork"

#Data parameters
image_size = 28*28
n_class = 10

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [2]:
#Train parameters
learning_rate = 0.1
total_steps = 200
batch_size = 64
conv_filters = [64, 32]
dense_units = [128]

X = tf.placeholder(tf.float32, [None, image_size], name='InputImage')
Y = tf.placeholder(tf.float32, [None, n_class], name='Label')

In [3]:
def conv_layer(self, output_ch, filter_size, stride, scope):
    with tf.name_scope(scope):
        input_ch = int(self.shape[-1])
        filters = tf.Variable(tf.random_normal([filter_size, filter_size, input_ch, output_ch]), name='filters')
        self = tf.nn.conv2d(self, filters, strides=[1, stride, stride, 1], padding='SAME')
        b = tf.Variable(tf.random_normal([output_ch]), name='bias')
        self = tf.nn.bias_add(self, b)
    return self

def maxpool_layer(self, ksize, stride, scope):
    with tf.name_scope(scope):
        self = tf.nn.max_pool(self, [1, ksize, ksize, 1], [1, stride, stride, 1], padding='SAME')
    return self

def dense_layer(self, next_units, scope):
    with tf.name_scope(scope):
        now_units = int(self.shape[1])
        W = tf.Variable(tf.random_normal([now_units, next_units], dtype=tf.float32), name='weight')
        b = tf.Variable(tf.random_normal([next_units]), name='bias')
        self = tf.add(tf.matmul(self, W), b)
    return self
    
def build_cnn(input_layer):
    with tf.name_scope('CNN'):
        net = conv_layer(input_layer, conv_filters[0], 3, 1, 'Conv_layer1')
        net = maxpool_layer(net, 3, 3, 'Max_layer1')
        net = conv_layer(net, conv_filters[1], 3, 1, 'Conv_layer2')
        net = maxpool_layer(net, 3, 3, 'Max_layer2')
    return net

def flatten_layer(input_net):
    with tf.name_scope('Flatten'):
        product = np.prod(np.asarray(input_net.shape.as_list()[1:]))
        net = tf.reshape(input_net, [-1, product])
    return net
        
def build_dnn(input_net):
    with tf.name_scope('DNN'):
        net = dense_layer(input_net, dense_units[0], 'Dense_layer1')
        net = dense_layer(net, n_class, 'Dense_layer2')
    return net

In [4]:
with tf.name_scope('Model'):
    input_layer = tf.reshape(X, [-1, 28, 28, 1])
    cnn = build_cnn(input_layer)
    cnn = flatten_layer(cnn)
    fcnn = build_dnn(cnn)

In [5]:
with tf.name_scope('Loss'):
    loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=fcnn, labels=Y))
with tf.name_scope("Adam"):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss_op)
with tf.name_scope('Accuracy'):
    correct_pred =  tf.equal(tf.argmax(fcnn, 1), tf.argmax(Y, 1))
    accuracy_op = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.global_variables_initializer()
input_image = tf.reshape(X, [-1, 28, 28, 1])
tf.summary.image("input_image", input_image)
tf.summary.scalar('loss', loss_op)
tf.summary.scalar('accuracy', accuracy_op)
merge_summary_op = tf.summary.merge_all()

In [6]:
with tf.Session() as sess:
    sess.run(init)
    train_writer = tf.summary.FileWriter(tb_logdir + '/train', graph=sess.graph)
    test_writer = tf.summary.FileWriter(tb_logdir + '/test')
    for step in range(total_steps):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        summary, train_acc, _ = sess.run([merge_summary_op, accuracy_op, train_op], feed_dict={X:batch_x, Y:batch_y})
        train_writer.add_summary(summary, step)
        summary, test_acc = sess.run([merge_summary_op, accuracy_op], feed_dict={X:mnist.test.images[:3000], Y:mnist.test.labels[:3000]})
        test_writer.add_summary(summary, step)
        if step%20 == 0:
            print("step: {}  train_acc:{:.2f}%  test accuracy:{:.2f}%".format(step, train_acc*100, test_acc*100))
    print('Training Complete!')

step: 0  train_acc:9.38%  test accuracy:9.57%
step: 20  train_acc:31.25%  test accuracy:32.67%
step: 40  train_acc:76.56%  test accuracy:71.43%
step: 60  train_acc:81.25%  test accuracy:82.23%
step: 80  train_acc:82.81%  test accuracy:83.03%
step: 100  train_acc:87.50%  test accuracy:88.67%
step: 120  train_acc:90.62%  test accuracy:88.50%
step: 140  train_acc:89.06%  test accuracy:88.63%
step: 160  train_acc:85.94%  test accuracy:90.40%
step: 180  train_acc:95.31%  test accuracy:91.60%
Training Complete!
